# Playgroung Augut 2021 
# LightGBM + Random search + k-fold Cross Validation

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import mean

# Load Data

In [ ]:
sample_subm = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
X_test0 = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv', index_col='id')
X_train0 = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv', index_col='id')

# EDA

In [ ]:
X_train0.describe()

In [ ]:
#Check correlations
X_train1 = pd.DataFrame(X_train0)

c = X_train1.corr().abs()

s = c.unstack()
so = s.sort_values(kind="quicksort", ascending=False)

print(so[100:150])

In [ ]:
# Visualize relevant variables and outliers in Train set
sns.pairplot(X_train0[['loss', 'f25', 'f52', 'f13', 
                    'f18', 'f2', 'f19','f77']])

I've seen it before, there's not much to do here... let the ML Boost model do the job.

# ML Model

**Train and Test Split - this is not done, because it is incldued the cross validation procedure!**

In [ ]:
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

X = X_train0.drop(['loss'], axis = 1)
y = X_train0.loss

print("Ok")
#train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=82)
#train_data = lightgbm.Dataset(train_X, label=train_y)
#test_data = lightgbm.Dataset(test_X, label=test_y)

# LightGBM Model

**Random Search for hyperparameter tunning.** <br/>
It is possible to Run a couple of rounds, reducing the variable space according to the primarly results. <br/>
**Makes use of k-fold cross validation.**

In [ ]:
import lightgbm

pp = 0
n_b_r = 0
e_s_r = 0

#np.random.seed(seed=68)

#Set the minimum error arbitrarily large
minr = 9999999999999
#How many runs to perform using randomly selected hyperparameters
iterations = 2 #50
seeds = 2 #3
for i in range(iterations):
    print('ITERATION: ', i+1)
    try:
        params = {} #initialize parameters
        params['boosting_type'] = np.random.choice(['gbdt']) #, 'goss']) #, 'dart'])  #'boosting' 
        params['metric'] = 'RMSE'
        params['learning_rate'] = np.random.uniform(0.0169, 0.0175)  #1st ROUND - (0.01, 0.9)        
        params['num_leaves'] = np.random.randint(16, 20)  #1st ROUND - (10, 200)  #OLD -31
        params['min_data'] = np.random.randint(14, 18)  #1st ROUND - (5, 50)
        params['max_depth'] = np.random.randint(13,16)  #1st ROUND - (-1, 30)
        params['feature_fraction'] = np.random.uniform(0.130, 0.145)  #1st ROUND (0.05, 0.95) 
        params['bagging_fraction'] = np.random.uniform(0.77, 0.9)  #1st ROUND (0.05, 0.95) 
        params['bagging_freq'] = np.random.randint(15, 18)   #1st ROUND 
        its = np.random.randint(3020, 3100)   #1st ROUND (1000, 8000) 
        e_stop = np.random.randint(65, 80)   #1st ROUND (50, 500) 
        params['verbose'] = -1
        print(params, its, e_stop)     

        train_data = lightgbm.Dataset(X, label=y)  
        
        rms_j = []
        for j1 in range(seeds):
            modelLGBM = lightgbm.cv(params,
                                   train_data,
                                    num_boost_round=its,
                                    nfold=3,
                                    early_stopping_rounds=e_stop,
                                    seed=np.random.randint(1, 9999),
                                    verbose_eval = -1 )  #silent evaluations
            #print(modelLGBM)
            a = map(lambda x: np.mean(modelLGBM[x]), modelLGBM)
            rms0 = list(a)         
            rms_j.append(rms0[0])
        print(rms_j)
        rms_j_aux = 0
        for k in range(len(rms_j)):
            rms_j_aux += rms_j[k]
        rms3 = rms_j_aux / len(rms_j)
        print("RMS average: ", rms3)

        if rms3 < minr:
            minr = rms3
            pp = params 
            n_b_r = its
            e_s_r = e_stop
    except Exception as e: 
        print('***** Failed - ', e)
        
print("*" * 100)
print('Minimum RMS: ', minr)
print('parameters = ', pp)
print('num_boost_round =', n_b_r)
print('early_stopping_rounds = ', e_s_r)

**Check the "optimal" parameters.**

In [ ]:
import lightgbm

t_size = 0.25 # can be changed 

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=t_size)
train_data = lightgbm.Dataset(train_X, label=train_y)
test_data = lightgbm.Dataset(test_X, label=test_y)

parameters =  {'boosting_type': 'gbdt',    #'goss','dart'
               'metric': 'RMSE',
               'learning_rate': 0.01734225667880825, 
               'num_leaves': 17, 
               'min_data': 15,
               'max_depth': 14,
               'feature_fraction': 0.13301919009818083,  
               'bagging_fraction': 0.8647874600606483,  
               'bagging_freq': 16,  
               'verbose': -1}  #silent errors
#parameters = pp
n_b_r =  3070
e_s_r =  74

modelLGBM = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=n_b_r, 
                       early_stopping_rounds=e_s_r, 
                        verbose_eval = -1 )  #silent evaluations

y_pred2 = modelLGBM.predict(test_X)
preds3 = [round(value) for value in y_pred2]

# evaluate predictions
rms3 = mean_squared_error(test_y, preds3, squared=False)
print("RMS: ", rms3)
      
g2=plt.scatter(test_y, preds3)

# Final Predict

In [ ]:
preds1F = modelLGBM.predict(X_test0)

print(preds1F)

pd.DataFrame(preds1F).boxplot(return_type='axes')

# Submission 

In [ ]:
output = pd.DataFrame({'id': X_test0.index,
                      'loss': preds1F})
output.set_index('id', inplace=True)
print(len(output))
output.to_csv('submission.csv')